In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import os
#import cplex
import pandas as pd
from os.path import exists
from pathlib import Path
import logging
study_name = "ModelSEED2"
base_directory = "/Users/chenry/"#This needs to change to map to the shared folder in your home directory
BaseModule_path = base_directory+"/code/KBBaseModules"
ModelSEEDpy_path = base_directory+"/code/ModelSEEDpy"
Ontology_path = base_directory+"/code/cb_annotation_ontology_api/lib"
COBRAKBase_path = base_directory+"/code/cobrakbase"
ModelSEEDReconstruction_path = base_directory+"/code/KB-ModelSEEDReconstruction"
sys.path = [ BaseModule_path, ModelSEEDpy_path,COBRAKBase_path,Ontology_path,ModelSEEDReconstruction_path+"/lib"] + sys.path
from modelseedpy import MSPackageManager, MSModelUtil, MSBuilder, MSATPCorrection, MSGapfill,MSModelReport
from modelseedpy.core.mstemplate import MSTemplateBuilder
from ModelSEEDReconstruction.modelseedrecon import ModelSEEDRecon
from installed_clients.WorkspaceClient import Workspace
#You need to get a KBase dev token or pull a token from the narrative and put in ~/.kbase/token file
token = None
with open(str(Path.home()) + '/.kbase/token', 'r') as fh:
    token = fh.read().strip()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

python version 3.9.13
modelseedpy 0.2.2
cobrakbase 0.3.1


In [2]:
#Run this cell for prod
wsclient = Workspace("https://kbase.us/services/ws", token=token)
#wsclient = Workspace("https://appdev.kbase.us/services/ws", token=token)
msrecon = ModelSEEDRecon({"data":ModelSEEDReconstruction_path+"/data/"},ModelSEEDReconstruction_path,base_directory+"/workspace/"+study_name,token,{"Workspace":wsclient},None)

In [3]:
model = msrecon.kbase_api.get_from_ws("153020/GCF_000005845.2.RAST.Complte_using_Object")
msrep = MSModelReport()
msrep.build_report(model,base_directory+"/scratch/"+model.name+".html")


/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:172: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  model_summary_df.style
/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:186: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  gapfillings_analysis_df.style
/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:204: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  gapfilling_df.style
/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:265: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(model_summary_df_styled.render(escape=False))
/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:269: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(gapfillings_analysis_df_styled.render(escape=False))
/Users/chenry//code/ModelSEEDpy/modelseedpy/core/msmode

In [3]:
output = msrecon.build_metabolic_models({
    "workspace":None,
    "genome_refs":["151253/GCF_000005845.2.RAST"],
    "run_gapfilling":True,
    "atp_safe":True,
    "forced_atp_list":[],
    "gapfilling_media_list":["KBaseMedia/Complete"],
    "suffix":".Complete.mdl",
    "core_template":"custom",
    "core_template_ref":"151253/CoreBacV5.1",
    "gs_template":"custom",
    "gs_template_ref":"151253/GramNegModelTemplateV5_test",
    "template_reactions_only":True,
    "output_core_models":True,
    "save_report_to_kbase":False
})

INFO:kbbasemodules.basemodule:build_metabolic_models:{
    "workspace": null,
    "genome_refs": [
        "151253/GCF_000005845.2.RAST"
    ],
    "run_gapfilling": true,
    "atp_safe": true,
    "forced_atp_list": [],
    "gapfilling_media_list": [
        "KBaseMedia/Complete"
    ],
    "suffix": ".Complete.mdl",
    "core_template": "custom",
    "core_template_ref": "151253/CoreBacV5.1",
    "gs_template": "custom",
    "gs_template_ref": "151253/GramNegModelTemplateV5_test",
    "template_reactions_only": true,
    "output_core_models": true,
    "save_report_to_kbase": false
}
/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(

In [3]:
output = msrecon.gapfill_metabolic_models({
    "workspace":151253,
    "model_list":["151253/GCF_000005845.2.RAST.GF_adtl_atrbts"],
    "atp_safe":True,
    "forced_atp_list":[],
    "media_list":["151253/CO2_H2"],
    "templates":None,
    "minimum_objective":0.01,
#    "reaction_exlusion_list":['rxn00121_c0',],
     "reaction_exlusion_list":[],
    "default_objective":"bio1",
    "save_report_to_kbase":False
})

INFO:kbbasemodules.basemodule:build_metabolic_models:{
    "workspace": null,
    "genome_refs": [
        "68303/Escherichia_coli_K-12_MG1655.RAST"
    ],
    "run_gapfilling": true,
    "atp_safe": false,
    "forced_atp_list": [],
    "gapfilling_media_list": [
        "KBaseMedia/Carbon-D-Glucose"
    ],
    "suffix": ".mdl",
    "core_template": "auto",
    "gs_template": "auto",
    "gs_template_ref": null,
    "template_reactions_only": true,
    "output_core_models": true,
    "save_report_to_kbase": false
}
/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ServerError: JSONRPCError: -32500. Object Escherichia_coli_K-12_MG1655.RAST cannot be accessed: User chenry may not read workspace 68303
us.kbase.workspace.database.exceptions.InaccessibleObjectException: Object Escherichia_coli_K-12_MG1655.RAST cannot be accessed: User chenry may not read workspace 68303
	at us.kbase.workspace.database.PermissionsCheckerFactory.throwInaccessibleObjectException(PermissionsCheckerFactory.java:501)
	at us.kbase.workspace.database.PermissionsCheckerFactory.access$900(PermissionsCheckerFactory.java:31)
	at us.kbase.workspace.database.PermissionsCheckerFactory$ObjectPermissionsChecker.check(PermissionsCheckerFactory.java:333)
	at us.kbase.workspace.database.ObjectResolver.resolve(ObjectResolver.java:202)
	at us.kbase.workspace.database.ObjectResolver.<init>(ObjectResolver.java:79)
	at us.kbase.workspace.database.ObjectResolver.<init>(ObjectResolver.java:37)
	at us.kbase.workspace.database.ObjectResolver$Builder.resolve(ObjectResolver.java:592)
	at us.kbase.

In [4]:
model = msrecon.kbase_api.get_from_ws("68304/Escherichia_coli_K-12_MG1655.RAST.mdl")
mdlutl = MSModelUtil.get(model)
mdlutl.model.objective = "bio1"
media = msrecon.kbase_api.get_from_ws("KBaseMedia/Carbon-D-Glucose")
mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print(str(mdlutl.model.slim_optimize()))

0.0


In [4]:
genome = msrecon.get_msgenome("68303/Escherichia_coli_K-12_MG1655.RAST")
core_template = msrecon.get_template("core_template_sulfur3","NewKBaseModelTemplates")
template = msrecon.get_template("GramNegModelTemplateV4","NewKBaseModelTemplates")

In [5]:
mdl = MSBuilder(genome, template).build("Escherichia_coli_K-12_MG1655.RAST.mdl", '0', False, False)
mdl.genome = genome#Move into MSBuilder?
mdl.template = template#Move into MSBuilder?
mdlutl = MSModelUtil.get(mdl)
atpcorrection = MSATPCorrection(mdlutl,core_template,[],load_default_medias=True,max_gapfilling=10,gapfilling_delta=0,forced_media=[],default_media_path=msrecon.module_dir+"/data/atp_medias.tsv")
tests = atpcorrection.run_atp_correction()
for test in tests:
    print(test["media"].id+":"+str(test["threshold"]))
print("Gapfilling:",len(atpcorrection.cumulative_core_gapfilling))

Adding biomass!!


INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd08701 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!


Glc:1.2000000000005457
Pyr:0.6
Fum:0.30000000000013644
For/NO2:0.8000000000003638
For/NO3:3.899999999999454
Pyr/NO3:7.200000000001091
Ac/NO3:4.350000000000272
Glc/DMSO:1.2000000000010913
Glc/TMAO:1.2000000000010913
Pyr/DMSO:0.6
Pyr/TMAO:0.6
Pyr/SO3:0.6
Gapfilling: 0


In [5]:
msrecon.save_model(mdlutl,68304)

In [6]:
media = msrecon.kbase_api.get_from_ws("KBaseMedia/Carbon-D-Glucose",None)
tests = mdlutl.atputl.build_tests()
msgapfill = MSGapfill(mdlutl,[template],[],tests,blacklist=[])
mdlutl.gfutl.cumulative_gapfilling = []
gfresults = msgapfill.run_gapfilling(media,"bio1",0.01)
msgapfill.integrate_gapfill_solution(gfresults,mdlutl.gfutl.cumulative_gapfilling)
mdlutl.model.objective = "bio1"
mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print(media.id+":"+str(mdlutl.model.slim_optimize()))

modelseedpy.fbapkg.gapfillingpkg - CRITICAL - Model cannot achieve the minimum objective even before filtering!
CRITICAL:modelseedpy.fbapkg.gapfillingpkg:Model cannot achieve the minimum objective even before filtering!
DEBUG:modelseedpy.core.msmodelutl:Expansion started! Binary = True
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f9008caea00>, 'is_max_threshold': True, 'threshold': 1.2000000000000002, 'objective': 'rxn00062_c0'}


Objective before filtering: 0.0 ; min objective: 0.01


KeyboardInterrupt: 

In [ ]:
msrecon.save_model(mdlutl,params["workspace"])
mdlutl.model.objective = "bio1"
mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(None)
print("Complete:"+str(mdlutl.model.slim_optimize()))